In [ ]:
# Run this cell to set up your notebook
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
#import folium

# DIISE Data Set

## Rat Eradications

We get the data set from http://diise.islandconservation.org/ and are interested in seeing the global state of rat eradications. On the website we click "explore" and in the box on the left select "by invasive species" and click "Rodent" for the type of species. Next, click "run search". As we can see, we have data on 837 islands covering 12 species and 1128 eradications. Feel free to explore other species. 

The table that is displayed on the bottom of the screen is the data we will be working with. 

In [ ]:
#load the data
rats = pd.read_csv("DIISE_2018_query_UTF8.csv")
pd.options.display.max_columns = None
rats.head()

Look at the table above. What do you notice? What columns stand out to you?

*Type your answer here.*

## About the data:

Invasive species (such as rats, snakes, rabbits, and cats) are a serious threat to native plants and animals on islands. Islands are currently being threatened by the ongoing climate crisis and eliminating invasive species will help protect island ecosystems and prevent extinctions. 

The Database of Invasive Island Species Eradications collects data on all past and present invasive species eradications on islands. The majority of the data from this database is on invasive mammals. Data points include the specific island, species name, method, and outcome. 

Now, lets dive into the rats table.

## Cleaning the data:

In [ ]:
# How many rows and columns are in the rats table?
rats.shape

The shape function is used to find out how many rows and columns are in the data set. The first number represents the number of rows and the last number represents the number of columns. Thus, in the rats dataset, there are 1126 rows and 30 columns. 

In [ ]:
# Run this to see all 30 columns in the rats table
rats.columns

Let's look at the rats table again.

In [ ]:
rats.head()

Notice how the table has 30 columns and many NaN and None values. NaN and None values are put in when certain data is lacking. Look at the "Organization_Website" column. From the first five rows, the website column is all NaN values. However, let's see if there are any real websites in later rows.

In [ ]:
# Find rows that contain an actual website 
rats[~rats['Organization_Website'].isnull()]

Scroll through this table and notice that all rows have actual websites. In the bottom of the data set you will see that there are 206 rows. This means that out of 1126 total rows in the rats dataset, only 206 of these rows contain actual data. Let's delete this column since only a small percentage of the "Organization_Website" column contains actual data. 

In [ ]:
# drop the Organization_Website column
rats = rats.drop('Organization_Website', axis=1)
rats.head()

Let's see if we can drop any other columns. Look at the "Human Population" column. It seems like there are a lot of None values. Let's do the same thing we did with the "Organization_Website" column. 

In [ ]:
# Find how many None values are in the "Human Population" column
rats[(rats['Human Population'] != 'None') & (rats['Human Population'] != 'Unknown')]

As we can see above, there are only 138 rows without 'None' or 'Unknown values'. Since only a small percentage of the rows in the "Human Population" column contain actual data, let's drop this column as well.

In [ ]:
# drop the "Human Population" column
rats = rats.drop('Human Population', axis=1)
rats.head()

## Exploratory Data Analysis

Exploratory data analysis refers to the process of performing investigations on data to discover patterns, spot anomalies, or test hypotheses. Usually this includes creating graphs or running statistical analyses. 

In [ ]:
# Here is the rats data set again
rats.head()

Locate the "Primary Eradication Method" column. This column shows the primary eradication method for each particular eradication. Let's see which method is the most common. 

In [ ]:
rats.groupby('Primary Eradication Method')['OBJECTID'].nunique().sort_values(ascending=False)

In [ ]:
# create a histogram of the primary eradication methods
sns.histplot(data=rats, x='Primary Eradication Method', color='green') #the color parameter allows you to change the color of your graph
plt.title('Number Of Eradications For Each Method') #this adds a title to your graph

In [ ]:
# create a histogram of the densities of primary eradication methods for 
# Brown/Norway Rat, Black Rat, House Mouse, and Polynesian Rat 
temp = rats.loc[rats['Common Name'].isin(['Brown / Norway Rat', 'Black Rat', 'House Mouse', 'Polynesian Rat'])]
sns.histplot(data=temp, x='Primary Eradication Method', hue='Common Name', stat='density',multiple='dodge')
plt.title('Densities of Primary Eradication Methods')

As we can see above, the most popular primary eradication method is toxicant and the least popular is hunting. Now, let's find out which specific toxicant is most popular.

In [ ]:
# Find which primary toxicant is most used
rats.groupby('Primary Toxicant')['OBJECTID'].nunique().sort_values(ascending=False)

Looking above, the second most frequent primary toxicant is "Unknown". Clearly, there is not a toxicant called "unknown", so we can infer that this means there are missing data points, which were replaced with "Unknown". We want to remove these "Unknown" values to get a better picture of the most popular toxicants. 

In [ ]:
# clean the "Primary Toxicant" column by removing rows where the Primary Toxicant is "Unknown"
remove_nan = rats[~rats['Primary Toxicant'].isin(['Unknown'])]
remove_nan = remove_nan[remove_nan['Primary Toxicant'].notna()]
remove_nan.head()

Now, let's see the numbers for each toxicant.

In [ ]:
remove_nan.groupby('Primary Toxicant')['OBJECTID'].nunique().sort_values(ascending=False)

Next, create a horizontal histogram to show the number of primary toxicants used in eradications. 

In [ ]:
# To create a hosrizontal histogram, use the 'y=' parameter instead of 'x='
sns.histplot(data=remove_nan, y='Primary Toxicant', color='purple')
plt.title('Primary Toxicant Counts')

Now, you may be wondering if these eradications are sucessful or not. Luckily, there is a column called "Status (Eradication)", which shows whether or not the eradication was successful. 

In [ ]:
# Display the statuses of eradications along with the counts for each
rats.groupby('Status (Eradication)')['OBJECTID'].nunique().sort_values(ascending=False)

Looking above, a majority of the eradication attempts are successful. "Successful" means the rodent is no longer present on the island and to our knowledge has not returned. "Successful (Reinvaded)" means the eradication was once successful, but the rodent has returned to the island.  

Next, lets look at the number of eradications over time. 

Create a table that shows each year in the "Eradication End Date" column and the number of eradications per year. 

In [ ]:
# Make a table with one column as the eradication end year in ascending order, 
# and another column as the count 
drop_nan_years = rats[rats['Eradication End Date'].notna()]
group_drop_nan_years = drop_nan_years.groupby('Eradication End Date').agg(len).reset_index()

In [ ]:
# Create a line graph showing the number of eradications per year 
sns.lineplot(data=group_drop_nan_years, x='Eradication End Date', y='Common Name', color='red')
plt.ylabel('Number of Eradications')
plt.title('Number Of Eradications Per Year')

Now, let's look at the number of eradications over time. Since there are many regions, let's focus on the Caribbean, Mediterranean, Pacific Ocean - South West, and Pacific Ocean - South East. Below, create a line plot with eradication end date on the x-axis and number of eradications on the y-axis. Create overlapping line plots for each region. 

In [ ]:
# only look at specific regions listed above
specific_regions = rats[rats['Region'].isin(['Caribbean', 'Mediterranean', 'Pacific Ocean - South West', 'Pacific Ocean - South East'])]

# group by Region and Eradication End Date 
grouped_specific_regions = specific_regions.groupby(['Region', 'Eradication End Date']).agg(len)

sns.lineplot(data=grouped_specific_regions, x='Eradication End Date', y='OBJECTID', hue='Region')

# Label your axes and title your graph 
plt.title('Number Of Eradications Per Year Per Region')
plt.ylabel('Number of Eradications')

Locations of eradication sites on a map.

In [ ]:
#m = folium.Map(location=[45.5236, -122.6750])


## Try it yourself!

Above, we walked you through analysis on rat eradications from the DIISE dataset. We looked at primary eradication methods and created a visualization to show the number of eradications over time for a few specific regions. Now, it's your turn to do your own exploratory data analysis. 

Go to this link: http://diise.islandconservation.org/ and click "explore". In the search box on the top left, click "...by invasive species" and you'll see a dropdown bar under "Type". Click the dropdown arrow and explore the different types of invasive species. To view the data on each species, click "Run Search". 

Find a species that interests you and that has a good amount of data to work with. We recommend looking at ungulates or mice. When you have chosen the species you want to explore, click the export CSV button on the top bar (on the bar of icons at the top left, it is the fourth from the left). 

Next, you're ready to upload your data to Jupyer notebook! In this part of this assignment, we will provide some starter code. Fill in the missing code wherever you see "...". We will be replicating the analysis we did above, so if you get stuck look at the code we provided for rat eradications above. 

In [ ]:
#load the data
... = pd.read_csv("...") # put the file name in the parenthesis
pd.options.display.max_columns = None
....head() 

Scroll through your data. Do you need to drop any rows or columns? You can drop columns using the .drop() function. If you need help, look at the code we provided above. Drop rows by isolating the rows you want to keep. Again, we've provided code to do this above. 

In [ ]:
# Drop any rows or columns here
# Feel free to add more cells as needed. To add a cell, click the plus sign button on the top of this page. 

Now, perform your own exploratory data analysis. First, let's look at the primary eradication methods for your species. 

In [ ]:
# show the primary eradication methods and the counts for each
....groupby('Primary Eradication Method')['OBJECTID'].nunique().sort_values(ascending=False)

In [ ]:
# create a histogram of the primary eradication methods
sns.histplot(data=..., x='Primary Eradication Method', color='...') 
plt.title('...') 

Next, let's look at the status of eradications for your species of choice. 

In [ ]:
# Display the statuses of eradications along with the counts for each
....groupby('Status (Eradication)')['OBJECTID'].nunique().sort_values(ascending=False)

Does anything about the data stand out to you?

Type your answer here.

Lastly, let's look at the number of eradications over time for your species. 

In [ ]:
# Make a table with one column as the eradication end year in ascending order, 
# and another column as the count 
drop_nan_years2 = ...[...['Eradication End Date'].notna()]
group_drop_nan_years2 = drop_nan_years2.groupby('Eradication End Date').agg(len).reset_index()

In [ ]:
# Create a line graph showing the number of eradications per year 
sns....(data=group_drop_nan_years2, x='...', y='Common Name', color='...')
plt.ylabel('...')
plt.title('...')

Now, let's look at eradication counts over time for a few specific regions. You can choose which regions you want to look at. 

In [ ]:
# choose specific regions 
specific_regions2 = ...[...['Region'].isin(['...', '...', '...', '...'])]

# group by Region and Eradication End Date 
grouped_specific_regions2 = specific_regions2.groupby(['...', '...']).agg(len)

sns....(data=grouped_specific_regions2, x='...', y='OBJECTID', hue='Region')

# Label your axes and title your graph 
plt.title('...')
plt.ylabel('...')